# Effecient Nets
use EfficientNet models for real-time applications using TensorFlow Lite. TensorFlow Lite is designed to optimize models for deployment on edge devices, providing smaller model sizes and faster inference times. EfficientNet models, especially the smaller variants like B0-B3, are well-suited for deployment using TensorFlow Lite on devices with limited computational resources, such as smartphones, Raspberry Pi, or IoT devices.

Steps to Use EfficientNet with TensorFlow Lite
Convert EfficientNet Model to TensorFlow Lite Format:

You need to convert the TensorFlow/Keras EfficientNet model to a .tflite format.
Use TensorFlow’s tf.lite.TFLiteConverter to convert the trained model.
Quantization:

Apply quantization techniques (e.g., post-training quantization) to further reduce the model size and improve inference speed.
Supported quantization types include float16, int8, and dynamic range.
Deploy on Edge Device:

Deploy the .tflite model on your target device, such as a Raspberry Pi or Android smartphone.
Use TensorFlow Lite’s interpreter to load and run inference.

In [ ]:
import tensorflow  
import tflite 
print(tensorflow.__version__ , tflite.__version__)

2.17.0 2.10.0


In [2]:
# sample code to generate data generators
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import keras_tuner as kt

2024-10-16 15:35:43.148404: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-16 15:35:43.159228: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 15:35:43.170458: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 15:35:43.173865: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 15:35:43.183816: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# General Hyperparameters
input_shape = (300, 300, 3)         
num_classes = 2                       
dropout_rate = 0.4                    

# Model Architecture Hyperparameters
dense_units_1 = 256                 
dense_units_2 = 128                 

# Data Augmentation Hyperparameters
rescale_value = 1./255             
rotation_range = 20                 
width_shift_range = 0.2
height_shift_range = 0.2              
shear_range = 0.2                  
zoom_range = 0.2                     
horizontal_flip = True            
fill_mode = 'nearest'              

# Training Hyperparameters
batch_size = 15                      
epochs = 10                               

# Callbacks Hyperparameters
reduce_lr_factor = 0.2             
reduce_lr_patience = 3            
min_lr = 1e-6                     
early_stopping_patience = 5      

# # optimizerAdam = 'adam'
# optimizerAdam = AdamW(
#     learning_rate=0.001,
#     weight_decay=0.004,
#     beta_1=0.9,
#     beta_2=0.999,
#     epsilon=1e-07,
#     amsgrad=False,
#     clipnorm=None,
#     clipvalue=None,
#     global_clipnorm=None,
#     use_ema=False,
#     ema_momentum=0.99,
#     ema_overwrite_frequency=None,
#     loss_scale_factor=None,
#     gradient_accumulation_steps=None,
#     name='adamw',
# )


2024-10-16 15:35:52.175121: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 419 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:4a:00.0, compute capability: 8.0
2024-10-16 15:35:52.177170: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22170 MB memory:  -> device: 1, name: NVIDIA A30, pci bus id: 0000:61:00.0, compute capability: 8.0
2024-10-16 15:35:52.179118: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 22170 MB memory:  -> device: 2, name: NVIDIA A30, pci bus id: 0000:ca:00.0, compute capability: 8.0
2024-10-16 15:35:52.181307: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 22170 MB memory:  -> device: 3, name: NVIDIA A30, pci bus id: 0000:e1:00.0, compute capability: 8.0


In [4]:
def lr_schedule(epoch, lr):
    if epoch > 5:
        lr = lr * 0.1  # Reduce learning rate after 5 epochs
    return lr

lr_callback = LearningRateScheduler(lr_schedule)

In [5]:
input_shape = (300, 300, 3)

optimizerAdam = AdamW(
    learning_rate=0.005,
    weight_decay=0.004,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    loss_scale_factor=None,
    gradient_accumulation_steps=None,
    name='adamw',
)

base_model = EfficientNetB3(
    include_top=False,            
    weights='imagenet',        
    input_shape=input_shape,     
    pooling='avg'                
)


model = models.Sequential([
    base_model,   
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.2),                              
    layers.Dense(256, activation='relu'),          
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),                         
    layers.Dense(2, activation='softmax')           
])


model.compile(
    optimizer=optimizerAdam,
    loss='categorical_crossentropy',           
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb3 (Functional)     │ (None, 1536)           │    10,783,535 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       786,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,734,961 (44.77 MB)

 Trainable params: 11,647,658 (44.43 MB)

 Non-trainable params: 87,303 (341.03 KB)

In [6]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

checkpoint = ModelCheckpoint(
    'best_model.keras', monitor='val_loss', save_best_only=True, save_weights_only=False
)

In [ ]:
# Training
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    '/home/research/Kilsar_Sentinal/EfficientNet data/splitted_data/train',
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    '/home/research/Kilsar_Sentinal/EfficientNet data/splitted_data/val',
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical'
)

Found 7996 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size,
    callbacks=[reduce_lr, early_stopping, checkpoint, lr_callback] 
)

/home/research/Kilsar_Sentinal/.venv/lib64/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1728520856.854215 1466927 service.cc:146] XLA service 0x7f648c002ee0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728520856.854246 1466927 service.cc:154]   StreamExecutor device (0): NVIDIA A30, Compute Capability 8.0
I0000 00:00:1728520856.854249 1466927 service.cc:154]   StreamExecutor device (1): NVIDIA A30, Compute Capability 8.0
I0000 00:00:1728520856.854250 1466927 service.cc:154]   StreamExecutor device (2): NVIDIA A30, Compute Capability 8.0
I0000 00:00:1728520856.854252 1466927 service.cc:154]   StreamExecutor device (3): NVIDIA A30, Compute Capability 8.0
2024-10-09 19:40:57.793202: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-09 19:41:01.248394: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902
2024-10-09 19:41:05.771292: I external/local_xla/xla/stre

 45/249 ━━━━━━━━━━━━━━━━━━━━ 53s 261ms/step - accuracy: 0.7489 - loss: 0.7458

2024-10-09 19:42:04.299355: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38409', 36 bytes spill stores, 40 bytes spill loads

2024-10-09 19:42:04.357272: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38143', 28 bytes spill stores, 28 bytes spill loads

2024-10-09 19:42:04.373895: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38407', 24 bytes spill stores, 24 bytes spill loads

2024-10-09 19:42:04.390944: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38143', 196 bytes spill stores, 196 bytes spill loads

E0000 00:00:1728520938.364898 1466930 gpu_timer.cc:183

...

249/249 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.8438 - loss: 0.4583 - val_accuracy: 0.9375 - val_loss: 0.6744 - learning_rate: 0.0050
Epoch 3/10


2024-10-09 19:43:51.546725: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


249/249 ━━━━━━━━━━━━━━━━━━━━ 68s 273ms/step - accuracy: 0.8102 - loss: 0.4390 - val_accuracy: 0.8100 - val_loss: 2.2421 - learning_rate: 0.0050
Epoch 4/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 166us/step - accuracy: 0.8438 - loss: 0.4391 - val_accuracy: 0.7500 - val_loss: 3.0821 - learning_rate: 1.0000e-03
Epoch 5/10


2024-10-09 19:44:59.963639: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


249/249 ━━━━━━━━━━━━━━━━━━━━ 68s 274ms/step - accuracy: 0.8213 - loss: 0.3973 - val_accuracy: 0.8100 - val_loss: 0.7882 - learning_rate: 1.0000e-03
Epoch 6/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 162us/step - accuracy: 0.8438 - loss: 0.2463 - val_accuracy: 0.7500 - val_loss: 1.0336 - learning_rate: 1.0000e-03


To unfreez layer weights

In [ ]:
# Freeze the base model layers
base_model.trainable = False

# Initial training with frozen base model
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size,
    callbacks=[reduce_lr, early_stopping, checkpoint]
)

# Unfreeze base model and fine-tune with a lower learning rate
base_model.trainable = True
model.compile(optimizer=AdamW(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history_finetune = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size,
    callbacks=[reduce_lr, early_stopping, checkpoint]
)


Epoch 1/5
249/249 ━━━━━━━━━━━━━━━━━━━━ 69s 275ms/step - accuracy: 0.8637 - loss: 0.3334 - val_accuracy: 0.8105 - val_loss: 0.8046 - learning_rate: 1.0000e-03
Epoch 2/5
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 172us/step - accuracy: 0.8438 - loss: 0.3316 - val_accuracy: 0.7500 - val_loss: 0.8468 - learning_rate: 1.0000e-03
Epoch 3/5
249/249 ━━━━━━━━━━━━━━━━━━━━ 68s 271ms/step - accuracy: 0.8652 - loss: 0.3239 - val_accuracy: 0.8105 - val_loss: 0.5964 - learning_rate: 1.0000e-03
Epoch 4/5
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 165us/step - accuracy: 0.9062 - loss: 0.2383 - val_accuracy: 0.6875 - val_loss: 0.8476 - learning_rate: 1.0000e-03
Epoch 5/5
249/249 ━━━━━━━━━━━━━━━━━━━━ 68s 273ms/step - accuracy: 0.8823 - loss: 0.2917 - val_accuracy: 0.8100 - val_loss: 0.6549 - learning_rate: 1.0000e-03
Epoch 1/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 150s 394ms/step - accuracy: 0.8585 - loss: 0.3232 - val_accuracy: 0.8760 - val_loss: 0.3168 - learning_rate: 1.0000e-05
Epoch 2/10
  1/249 ━━━━━━━━━━━━━━━━━━━━ 38s 156ms/st

2024-10-09 20:37:38.624046: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


249/249 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9062 - loss: 0.2842 - val_accuracy: 1.0000 - val_loss: 0.1659 - learning_rate: 1.0000e-05
Epoch 3/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 68s 274ms/step - accuracy: 0.8660 - loss: 0.3110 - val_accuracy: 0.8458 - val_loss: 0.3673 - learning_rate: 1.0000e-05
Epoch 4/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 165us/step - accuracy: 0.9062 - loss: 0.2650 - val_accuracy: 0.8750 - val_loss: 0.4229 - learning_rate: 1.0000e-05
Epoch 5/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 68s 272ms/step - accuracy: 0.8789 - loss: 0.2945 - val_accuracy: 0.8140 - val_loss: 0.5905 - learning_rate: 1.0000e-05
Epoch 6/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 166us/step - accuracy: 0.8750 - loss: 0.2435 - val_accuracy: 0.7500 - val_loss: 0.8426 - learning_rate: 2.0000e-06
Epoch 7/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 68s 273ms/step - accuracy: 0.8756 - loss: 0.3009 - val_accuracy: 0.8967 - val_loss: 0.2659 - learning_rate: 2.0000e-06


In [ ]:
def build_model(hp):
    # Build a model with hyperparameters to tune
    base_model = EfficientNetB3(include_top=False, weights='imagenet', input_shape=input_shape, pooling='avg')
    model = models.Sequential([
        base_model,
        layers.Dense(hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        layers.Dense(hp.Int('units2', min_value=32, max_value=256, step=32), activation='relu'),
        layers.Dropout(hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)),
        layers.Dense(2, activation='softmax')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-3, 1e-4, 1e-5])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Create a tuner and search for the best hyperparameters
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner',
    project_name='efficientnet_tuning'
)
tuner.search(train_generator, validation_data=val_generator, epochs=10)


## Initial Model setup

In [ ]:
# Initial Model
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB3


input_shape = (300, 300, 3)

base_model = EfficientNetB3(
    include_top=False,            
    weights='imagenet',        
    input_shape=input_shape,     
    pooling='avg'                
)


model = models.Sequential([
    base_model,                                 
    layers.Dense(256, activation='relu'),          
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),                         
    layers.Dense(2, activation='softmax')           
])


model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',           
    metrics=['accuracy']
)

# Summary of the custom model
model.summary()

# Training
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    '/home/research/Kilsar_Sentinal/EfficientNet data/splitted_data/train',
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    '/home/research/Kilsar_Sentinal/EfficientNet data/splitted_data/val',
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical'
)

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size
)

To save converted model

In [ ]:
model.save('/home/research/Kilsar_Sentinal/suryansh/src/models/tf models/model-2.keras')

In [ ]:

converter = tf.lite.TFLiteConverter.from_saved_model('efficientnet_b0_saved_model')
# Optional: apply optimizations like quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Convert the model
tflite_model = converter.convert()

# Save the converted .tflite model
with open('efficientnet_b0.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
import tensorflow as tf

# Load the EfficientNetB3 model
model = tf.keras.applications.EfficientNetB3(
    include_top=True,
    weights='imagenet',  
    input_shape=(300, 300, 3) 
)

model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB3

# Define input shape based on the dataset (e.g., 300x300 RGB images)
input_shape = (300, 300, 3)

# Load the EfficientNet-B3 model with custom settings
base_model = EfficientNetB3(
    include_top=False,            # Remove the default top layer to add custom layers
    weights='imagenet',           # Use pre-trained ImageNet weights
    input_shape=input_shape,      # Set input shape
    pooling='avg'                 # Apply global average pooling
)

# Freeze the base model layers to retain pre-trained features
base_model.trainable = False

# Create a new model on top of the base model
model = models.Sequential([
    base_model,                                    # EfficientNetB3 base model
    layers.Dense(256, activation='relu'),          # Custom fully connected layer for feature extraction
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),                           # Dropout layer for regularization
    layers.Dense(2, activation='softmax')          # Output layer with 2 classes (for 2 gun types)
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',               # Use categorical cross-entropy for multi-class classification
    metrics=['accuracy']
)

# Summary of the custom model
model.summary()


In [ ]:
import kerastuner as kt
from tensorflow.keras.optimizers import Adam

train_dir = '/home/research/Kilsar_Sentinal/EfficientNet data/splitted_data/train'

# enabling gpu
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth set successfully.")
    except RuntimeError as e:
        print(f"Error setting memory growth: {e}")


# Define a function for model building that integrates with Keras Tuner
def build_model(hp):
    base_model = EfficientNetB3(
        include_top=False,
        weights='imagenet',
        input_shape=(300, 300, 3),
        pooling='avg'
    )
    
    # Create the model
    model = models.Sequential([
        base_model,
        layers.Dense(hp.Int('dense_units_1', min_value=128, max_value=512, step=128), activation='relu'),
        layers.Dense(hp.Int('dense_units_2', min_value=64, max_value=256, step=64), activation='relu'),
        layers.Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.4, step=0.1)),
        layers.Dense(2, activation='softmax')
    ])
    
    # Compile the model
    model.compile(
        optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create a tuner instance
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='tuner_dir',
    project_name='hyperparam_tuning'
)

# Prepare training data
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical'
)

# Perform the search
tuner.search(train_generator, epochs=10, validation_data=train_generator)

# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps.values}")

Error setting memory growth: Physical devices cannot be modified after being initialized
Reloading Tuner from tuner_dir/hyperparam_tuning/tuner0.json
Found 7996 images belonging to 2 classes.

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
512               |128               |dense_units_1
256               |128               |dense_units_2
0.4               |0.2               |dropout_rate
0.001             |0.0001            |learning_rate

Epoch 1/10


2024-10-09 20:43:40.558117: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36329', 4 bytes spill stores, 4 bytes spill loads

2024-10-09 20:43:40.712028: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36079', 8 bytes spill stores, 8 bytes spill loads

2024-10-09 20:43:40.992237: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36079', 4 bytes spill stores, 4 bytes spill loads

2024-10-09 20:43:41.089026: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36079', 168 bytes spill stores, 168 bytes spill loads

2024-10-09 20:44:03.302548: I external/local_xla/xla/stream_

167/250 ━━━━━━━━━━━━━━━━━━━━ 12s 146ms/step - accuracy: 0.8853 - loss: 0.3005

2024-10-09 20:44:35.845524: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36329', 36 bytes spill stores, 36 bytes spill loads

2024-10-09 20:44:36.130685: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36343', 24 bytes spill stores, 24 bytes spill loads

2024-10-09 20:44:36.150607: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36345', 36 bytes spill stores, 40 bytes spill loads

2024-10-09 20:44:36.180024: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36079', 28 bytes spill stores, 28 bytes spill loads

2024-10-09 20:44:36.213519: I external/local_xla/xla/str

250/250 ━━━━━━━━━━━━━━━━━━━━ 144s 351ms/step - accuracy: 0.8944 - loss: 0.2819 - val_accuracy: 0.8098 - val_loss: 0.4993
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.9532 - loss: 0.1419